In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_fresh_daily 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) 
AS
WITH 
TAB02 AS (
    SELECT *
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active
),
ASRT_LIST AS (
    SELECT DISTINCT 
        a.store_id, 
        a.product_id
    FROM TAB02 a
),
SITE_ARTICLE AS (
    SELECT 
        a.store_id, 
        a.product_id
    FROM ASRT_LIST a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
    WHERE b.business_unit IN ('1500', '2000', '6000') 
        AND b.store_status_id <> 'CLOSED' 
        AND UPPER(b.store_name) NOT LIKE '[BLOCK]%'
        AND UPPER(b.store_name) NOT LIKE 'OUTLET%' 
        AND UPPER(b.store_name) NOT LIKE '%HUB%' 
        AND UPPER(b.store_name) NOT LIKE '%ONLINE%'
), 
MEATDELI AS (
    SELECT * EXCEPT (rn)
        FROM (
            SELECT 
                ROW_NUMBER() OVER (PARTITION BY store_id, product_id ORDER BY proc_date DESC) rn,
                store_id,
                product_id,
                'MEATDELI' order_channel,
                base_uom,
                mdq,
                proc_date 
            FROM {catalog_name}.udp_wcm_silver_dim.eton_active_store_sku_meatdeli 
            WHERE DATE(proc_date) > '2021-06-20'
        ) WHERE rn=1
)
, _3F AS (
    SELECT * EXCEPT (rn)
        FROM (
            SELECT 
                ROW_NUMBER() OVER (PARTITION BY store_id, product_id ORDER BY proc_date DESC) rn,
                store_id,
                product_id,
                '3F' order_channel,
                base_uom,
                mdq,
                proc_date 
            FROM {catalog_name}.udp_wcm_silver_dim.eton_active_store_sku_3f  
            WHERE DATE(proc_date) > '2021-06-20'
        ) WHERE rn=1
)
, FRUIT AS (
    SELECT * EXCEPT (rn)
        FROM (
            SELECT 
                ROW_NUMBER() OVER (PARTITION BY store_id, product_id ORDER BY proc_date DESC) rn,
                store_id,
                product_id,
                'FRUIT' order_channel,
                base_uom,
                mdq,
                proc_date 
            FROM {catalog_name}.udp_wcm_silver_dim.eton_active_store_sku_fruit 
            WHERE DATE(proc_date) > '2021-06-20'
        ) WHERE rn=1
)
, CUTOFF AS (
    SELECT DISTINCT
        store_id, 
        product_id,
        '' AS order_channel,
        base_uom,
        mdq,
        CAST(DATE_ADD(start_date, 1) AS TIMESTAMP) proc_date
    FROM {catalog_name}.udp_wcm_silver_dim.minstock_eton_fresh_scd_cutoff 
)
SELECT  * EXCEPT (rn, proc_date),
        CASE 
            WHEN a.mdq IS NULL THEN 'NOT AVAILABLE'
            WHEN a.mdq IS NOT NULL AND a.mdq <> 0 THEN 'AVAILABLE'
            WHEN a.mdq IS NOT NULL AND a.mdq = 0 THEN '0'
        END mdq_available
FROM    (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY store_id, product_id ORDER BY proc_date DESC) rn 
    FROM (
        SELECT * FROM MEATDELI
        UNION ALL
        SELECT * FROM _3F
        UNION ALL
        SELECT * FROM CUTOFF
        UNION ALL 
        SELECT * FROM FRUIT
    ) 
) a
JOIN    SITE_ARTICLE a1 USING (store_id, product_id)
WHERE   a.rn=1 
;
""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_fresh_daily_hist
(
calday DATE,
store_id STRING,
product_id STRING,
mdq DOUBLE
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.D_MDQ_FRESH_DAILY_HIST
SELECT DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AS calday, store_id, product_id, mdq FROM {catalog_name}.udp_wcm_gold_vcm_dwh.D_MDQ_FRESH_DAILY
;""")